# Introduction to GROMACS
This notebook provides a basic overview of using the GROMACS simulation engine to perform an energy minimization and basic molecular dynamics simulation.

To perform a simulation, GROMACS requires three key input files: 

 * The ".top" file that defines the system topology (bonds, angles, dihedrals) and their associated parameters 
 * The ".gro" file defines the coordinates and box size.  
 * The ".mdp" file that defines which simulation algorithms to use and their associated input parameters. 
 
    The structure of these files is relatively straight forward, and extensively documented elsewhere ( See the GROMACS manual: [top file](http://manual.gromacs.org/online/top.html), [gro file](http://manual.gromacs.org/online/gro.html), and [mdp file](http://manual.gromacs.org/online/mdp.html). In general, one does not need to generate the .top and .gro files by hand, as there are extensive  databases,  tools to facilitate conversion from common (e.g., the ```pdb2gmx``` tool in GROMACS to convert from PDB to ".gro" [(see the manual for more information)](http://manual.gromacs.org/programs/gmx-pdb2gmx.html)), and software such as [MoSDeF](http://github.com/mosdef-hub/) that will generate these files automatically. 

### Example, energy minimization of a hexane molecule
Let us consider a single hexane molecule; we have included "hexane.top" and "hexane.gro" in "hexane" directory of this tutorial.  The file "hexane.gro" can be easily viewed in a text editor or visualized with [VMD](https://www.ks.uiuc.edu/Research/vmd/).  As can be seen in the image below, this intial configuration is clearly not an energetically favorable state for the molecule. 

!["VMD rendering"](hexane/hexane.jpg)

Let us  consider performing an energy minimization using GROMACS. To perform the energy minimization, we'll need to create an .mdp file to tell GROMACS what to do. The contents of the "minim.mdp" file included in the "hexane" source directory is shown below, with comments describing each of the entries in the file; for more information on the commands in the .mdp option, see the [mdp options page](http://manual.gromacs.org/online/mdp_opt.html) and the [mdp file format ](http://manual.gromacs.org/online/mdp.html) pages in the manual.

```
; minim.mdp - used as input into grompp to generate em.tpr
integrator     = steep     ; Algorithm (steep = steepest descent minimization)
emtol          = 1000.0    ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep         = 0.01      ; Energy step size
nsteps         = 50000     ; Maximum number of (minimization) steps to perform
nstxout        = 1         ; output frequency for trajectory
nstlog         = 1         ; output frequency for log
; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist        = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme  = Verlet
ns_type        = simple    ; Method to determine neighbor list (simple, grid)
rvdw           = 1.0	   ; Short-range Van der Waals cut-off
pbc            = xyz       ; Periodic Boundary Conditions (yes/no)
coulombtype    = PME	   ; Long range solver  
fourierspacing = 0.16      ; grid size
```

Before we can use perform the energy minimazation, we must run the .mdp file through the ```grompp``` preprocessor to prepare the files for running.  This preprocessing step checks for errors and also provides warnings and useful feedback to properly prepare your simulation files.  This command takes, as input, the ".mdp" file, the ".gro" file and ".top" file and outputs a ".tpr" file. See [the page on grompp in the manual](http://manual.gromacs.org/programs/gmx-grompp.html) for more information.

In [1]:
%%bash 
gmx grompp -f hexane/minim.mdp -c hexane/hexane.gro -p hexane/hexane.top  -o hexane/em.tpr

Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.152 0.152 0.141
This run will generate roughly 150 Mb of data


                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

Next, we use the ```mdrun``` command to actually execute the script.

In [2]:
%%bash
gmx mdrun -v -deffnm hexane/em

                      :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS dev

This will generate a series of files. "em.gro" is the configuration from the last step of the simulation. "em.log" is the simulation log. "em.trr" is the configuration trajectory. 

If we visualize "em.gro" (or "em.trr") we can see the hexane molecule has energy minimized into the expected conformation.

!["Energy mimized hexane molecule"](hexane/hexane_em.jpg)

### Building a system of hexane molecules
To go beyond a single molecule, we can utilize the ```insert-molecules``` function of GROMACS which will construct a random configuration.  This command will take as input a ".gro" file defining the molecule, the target number of molecules, and the box size.  For example, let us create 5 copies of hexane.  We can use the initial configuration ("hexane.gro") as input, but it would make more sense to use the energy minimized molecule we just generated ("em.gro").  Note, lengths in GROMACS are given in nanometers (not angstroms). 

In [3]:
%%bash
gmx insert-molecules -ci hexane/em.gro -nmol 5 -box 5 5 5 -o hexane_box/hexane_box5.gro


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



                 :-) GROMACS - gmx insert-molecules, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMA

As before, we can easily visualize the "hexane_box.gro" file using VMD. 
!["System of 5 hexane molecules"](hexane_box/hexane5.jpg)

Let us increase the number of hexane molecules to 128 in order to create a bit more of a realistic system (VMD rendering show below). 
!["System of 128 hexane molecules"](hexane_box/hexane128.jpg)


In [4]:
%%bash
gmx insert-molecules -ci hexane/em.gro -nmol 128 -box 5 5 5 -o hexane_box/hexane_box128.gro


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



                 :-) GROMACS - gmx insert-molecules, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMA

### Running an MD simulation of 128 hexane molecules
With the basic structure built, we can now run the system (note, in most cases we would want to perform several stages to relax the system, but for simiplicity of simply demonstrating how to use the basic features of GROMACS, we will skip this step).  The "hexane_box" directory includes an .mdp file named "simple_run.mdp".  The contents of this file are shown below:

```
constraints         = none
integrator          = md
nsteps              = 100000
dt                  = 0.001

nstxout             = 10000
nstvout             = 0
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
rcoulomb            = 1.1
rvdw                = 1.1

coulombtype         = PME
fourierspacing      = 0.16

gen_vel             = no

tcoupl              = berendsen
tc-grps             = System
tau_t               = 1
ref_t               = 300

pcoupl              = berendsen
pcoupltype          = isotropic
tau_p               = 1
ref_p               = 1
compressibility     = 4.5e-5
```

Note, we will need to ensure that the ".top" file for hexane reflects the fact the system now has 128 molecules.  The  file "hexane_box128.top" is identical to the original "hexane.top" file, save for the last section:

```
[ molecules ]
; Compound       #mols
CH6                  128
```

Again, we will first have to run the .mdp file through the ```grompp``` preprocessor. Note, if you failed to update the number of molecules in the .top file, the preprocessor with provide the following, helfpul error:

```
Fatal error:
number of coordinates in coordinate file (hexane_box128.gro, 2560)
             does not match topology (hexane_box128.top, 20)
```

In [5]:
%%bash
gmx grompp -f hexane_box/simple_run.mdp -c hexane_box/hexane_box128.gro -p hexane_box/hexane_box128.top  -o hexane_box/rlx.tprc

Analysing residue names:
There are:   128      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K
Calculated rlist for 1x1 atom pair-list as 1.100 nm, buffer size 0.000 nm
Set rlist, assuming 4x4 atom pair-list, to 1.100 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 32x32x32, spacing 0.156 0.156 0.156
This run will generate roughly 4 Mb of data


                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

After the preprocessing, we can now run this simulation using the following command. Note, the output will not appear until the job has completed; to see the status in realtime, copy and pste the command (exclude the "%%bash" line) to your terminal.

In [7]:
%%bash
gmx mdrun -v -deffnm hexane_box/rlx

                      :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS dev

The "rlx.trr" file can be viewed in VMD; the final snapshot of the short simulation run in shown below.

!["final snapshot"](hexane_box/hexane128_sim.jpg)